In [ ]:
# Installiere benötigte Pakete
!pip install pandas requests lxml

In [ ]:
import pandas as pd
import requests
from datetime import datetime, timedelta

# Zeitraum festlegen
start_date = datetime(2025, 2, 1)
end_date = datetime(2025, 2, 5)

station = 'DOMODOSSOLA'
all_data = []

current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime('%d_%m_%Y')
    url = f'https://trainstats.altervista.org/speciali/stazioni/table.php?data={date_str}&n={station}'

    try:
        response = requests.get(url)
        response.raise_for_status()
        tables = pd.read_html(response.text)
        if tables:
            df = tables[0]
            df.insert(0, 'Datum', current_date.strftime('%Y-%m-%d'))
            all_data.append(df)
            print(f'Daten für {date_str} geladen.')
        else:
            print(f'Keine Tabelle für {date_str}.')
    except Exception as e:
        print(f'Fehler bei {date_str}: {e}')

    current_date += timedelta(days=1)

if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    final_df.columns = [col.strip().replace(' ', '_') for col in final_df.columns]
    final_df.to_csv('domodossola_zugfahrten.csv', index=False, encoding='utf-8')
    print(f'CSV gespeichert: domodossola_zugfahrten.csv (Zeilen: {len(final_df)})')
else:
    print('Keine Daten gefunden.')


In [ ]:
# CSV-Datei herunterladen
from IPython.display import FileLink
FileLink('domodossola_zugfahrten.csv')